In [1]:
# Crear puntos de entrada para spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Ejemplo DataFrame

In [4]:
mtcars = spark.read.csv(path='../../data/mtcars.csv',
                        sep=',',
                        encoding='UTF-8',
                        comment=None,
                        header=True, 
                        inferSchema=True)

## DataFrame para RDD
Un **DataFrame** se puede convertir fácilmente en un **RDD** llamando a  `pyspark.sql.DataFrame.rdd()` función Cada elemento en el RDD devuelto es un objeto **pyspark.sql.Row**. Una fila es una lista de pares clave-valor.

In [5]:
mtcars.rdd.take(2)

[Row(_c0='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4),
 Row(_c0='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4)]

Con un objeto RDD, podemos aplicar un conjunto de funciones de mapeo, tales como **map**, **mapValues**, **flatMap**, **flatMapValues** y muchos otros métodos que vienen de RDD.

In [6]:
mtcars_map = mtcars.rdd.map(lambda x: (x['_c0'], x['mpg']))
mtcars_map.take(5)

[('Mazda RX4', 21.0),
 ('Mazda RX4 Wag', 21.0),
 ('Datsun 710', 22.8),
 ('Hornet 4 Drive', 21.4),
 ('Hornet Sportabout', 18.7)]

In [7]:
mtcars_mapvalues = mtcars_map.mapValues(lambda x: [x, x * 10])
mtcars_mapvalues.take(5)

[('Mazda RX4', [21.0, 210.0]),
 ('Mazda RX4 Wag', [21.0, 210.0]),
 ('Datsun 710', [22.8, 228.0]),
 ('Hornet 4 Drive', [21.4, 214.0]),
 ('Hornet Sportabout', [18.7, 187.0])]

## RDD para DataFrame

Para convertir un RDD en un DataFrame, podemos usar la función `SparkSession.createDataFrame()`. Cada elemento en el RDD **tiene que ser a un Row object**.

Crear RDD

In [8]:
rdd_raw = sc.textFile('../../data/mtcars.csv')
rdd_raw.take(5)

[',mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb',
 'Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4',
 'Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4',
 'Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1',
 'Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1']

#### Grabar la primera línea en una variable

In [9]:
header = rdd_raw.map(lambda x: x.split(',')).filter(lambda x: x[1] == 'mpg').collect()[0]
header[0] = 'model'
header

['model',
 'mpg',
 'cyl',
 'disp',
 'hp',
 'drat',
 'wt',
 'qsec',
 'vs',
 'am',
 'gear',
 'carb']

#### Guardar el resto en un nuevo RDD

In [10]:
rdd = rdd_raw.map(lambda x: x.split(',')).filter(lambda x: x[1] != 'mpg')
rdd.take(2)

[['Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4'],
 ['Mazda RX4 Wag',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.875',
  '17.02',
  '0',
  '1',
  '4',
  '4']]

#### Conververtir elementos RDD a RDD Row objects

Primero definimos una función que toma una lista de nombres de columnas y una lista de valores y creamos una fila de pares clave-valor. **Dado que las teclas de un objeto Row son nombres de variables, no podemos simplemente pasar un diccionario a la función Row()**. Podemos pensar en un diccionario como una lista de argumentos y usar el `**` para descomprimir la lista de argumentos.

Por ejemplo.

In [11]:
from pyspark.sql import Row
my_dict = dict(zip(['a', 'b', 'c'], range(1, 4)))
Row(**my_dict)

Row(a=1, b=2, c=3)

#### Vamos a definir la función.

In [12]:
def list_to_row(keys, values):
    row_dict = dict(zip(keys, values))
    return Row(**row_dict)

#### Convertir elementos en objetos RDD

In [13]:
rdd_rows = rdd.map(lambda x: list_to_row(header, x))
rdd_rows.take(3)

[Row(am='1', carb='4', cyl='6', disp='160', drat='3.9', gear='4', hp='110', model='Mazda RX4', mpg='21', qsec='16.46', vs='0', wt='2.62'),
 Row(am='1', carb='4', cyl='6', disp='160', drat='3.9', gear='4', hp='110', model='Mazda RX4 Wag', mpg='21', qsec='17.02', vs='0', wt='2.875'),
 Row(am='1', carb='1', cyl='4', disp='108', drat='3.85', gear='4', hp='93', model='Datsun 710', mpg='22.8', qsec='18.61', vs='1', wt='2.32')]

Ahora podemos convertir el RDD en un DataFrame.

In [14]:
df = spark.createDataFrame(rdd_rows)
df.show(5)

+---+----+---+----+----+----+---+-----------------+----+-----+---+-----+
| am|carb|cyl|disp|drat|gear| hp|            model| mpg| qsec| vs|   wt|
+---+----+---+----+----+----+---+-----------------+----+-----+---+-----+
|  1|   4|  6| 160| 3.9|   4|110|        Mazda RX4|  21|16.46|  0| 2.62|
|  1|   4|  6| 160| 3.9|   4|110|    Mazda RX4 Wag|  21|17.02|  0|2.875|
|  1|   1|  4| 108|3.85|   4| 93|       Datsun 710|22.8|18.61|  1| 2.32|
|  0|   1|  6| 258|3.08|   3|110|   Hornet 4 Drive|21.4|19.44|  1|3.215|
|  0|   2|  8| 360|3.15|   3|175|Hornet Sportabout|18.7|17.02|  0| 3.44|
+---+----+---+----+----+----+---+-----------------+----+-----+---+-----+
only showing top 5 rows

